# Banc Statique <br>
### STAGE ANCHES <br>
Camille Urban <br>
05/07/2024

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import os

In [2]:
%matplotlib
%matplotlib

Using matplotlib backend: <object object at 0x00000201958A22F0>
Using matplotlib backend: TkAgg


# Paramètres

In [3]:
# SENSIBILITE CAPTEUR
# S_force_bs =       # bs pour banc statique, sensibilité en N/mV
G_force_bs = 900        # gain du banc de mesure

# cannal d'acquisition
# Cannal 2 

seuil = 150

In [4]:
dossier = '2024.05.30'
imag = '/image_'

# Fonctions

In [5]:
def bord_anche(image):
    points = []
    I = np.flipud(image)
    (H, W) = I.shape
    # print(H,W)
    plt.imshow(I, cmap='gray', extent=[0, W, H, 0])
    plt.axis('equal')
    plt.xlim(0, W)
    plt.ylim(0, H)
    plt.title("Cliquez sur les bords de l'anche")
    coord_anche = plt.ginput(2)
    plt.close()
    return coord_anche

In [6]:
# FONCTION DE ROTATION D'IMAGE 
# entrée : une image
# ginput : bord gauche et bord droit du bec
# sortie : angle de rotation, image J après rotation, coordonées des points selectionnés
def rotate(image, coord):
    # Calculer l'angle pour aligner l'image
    (x1, y1), (x2, y2) = coord
    rot_angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
    # Calculer le centre de l'image
    (h, w) = image.shape
    center = (w // 2, h // 2)

    # Créer une matrice de rotation centrale
    rot_mat = cv2.getRotationMatrix2D(center, rot_angle, 1.0)

    # Appliquer la rotation
    img_rot = cv2.warpAffine(image, rot_mat, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    return rot_mat, img_rot


In [7]:
# ROGNER IMAGE
# entrée : image, coordonées du bec (_,_, xy_bec = im_rotate(J))
def rogner(image, coord):
    # img_rogn = np.flipud(image)
    x_BG = coord[0][0]
    y_BG = coord[0][1] 
    x_BD = coord[1][0]
    y_BD = coord[1][1]
    
        # bornes 
    wmin = int(x_BG)-20     # width min
    wmax = int(x_BD)+20
    hmin = int(y_BG-50)     # height min
    hmax = int(y_BG+350)
    img_rogn = image[hmin:hmax, wmin:wmax] 
    
    return img_rogn

In [8]:
def deplacement_profil(image, seuil, coord):
    # s'assurer que I est bien en binaire
    _, I = cv2.threshold(image, seuil, 255, cv2.THRESH_BINARY)
    
    # coord = bord_anche(image)
        
    (x1, y1), (x2, y2) = map(lambda p: (int(p[0]), int(p[1])), coord)
    # initialisation matrices vides
    Xv = np.arange(x1, x2, dtype=int)   # vecteur de la taille de la largeur de l'image coupée
    Yv = []

# PROBLEME POUR GENERALISER A UNE TRANCHE ENTIERE
    for xv in Xv:
        Iv = I[:, xv].astype(float)   # vecteur qui contient une tranche de l'image largeur 1 (x) et hauteur de l'image (y)
        Iv_2 = np.round(np.gradient(Iv)).astype(int)     # Dérivée centrée (flottante) + Arrondir + convertir en entiers

        # seuillage trance par tranche
        smin = -30
        pos2 = np.where(Iv_2 < smin)[0][-1] if any(Iv_2 < smin) else None
        H = I.shape[0]
        Y = np.arange(1,H+1)
    
        if pos2 is not None:
            Yv.append(Y[pos2])
        else:
            Yv.append(np.nan)
    
    plot_fig = True
    if plot_fig :
        plt.figure()
        plt.imshow(image, cmap='gray')
        plt.plot(Xv, Yv, 'm.-', linewidth=0.1)
            
    return Xv, Yv



In [9]:
# I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE)
# _, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
# I = I.astype(np.float32)

# xy_anche = bord_anche(I)
# rot_mat, I_rot = rotate(I, xy_anche)
# I_rogn = rogner(I_rot, xy_anche)
# # plt.imshow(I_rogn, cmap='gray')


In [10]:
# coord = bord_anche(I_rogn)
# Xv, Yv = deplacement_profil(I_rogn, seuil, coord)

In [12]:
plt.close('all')

# Initialisation sur une image

# Lecture de l'image en nuance de gris, mise en noir et blanc
I = cv2.imread(dossier + imag + '0_0.jpg', cv2.IMREAD_GRAYSCALE)
_, I = cv2.threshold(I, seuil, 255, cv2.THRESH_BINARY)
I = I.astype(np.float32)

xy_anche = bord_anche(I)
rot_mat, I_rot = rotate(I, xy_anche)
I_rogn = rogner(I_rot, xy_anche)
coord_rogn = bord_anche(I_rogn)

Ys = 13
Zs = 21
pas_z = 0.05 #mm
(w, h) = I_rogn.shape

# Import data
for y in range(Ys):
    with open(dossier + f'/profil{y}.txt', 'w') as file:
        for z in range(Zs):
            img_path = dossier + imag + f'{y}_{z}.jpg'
            if not os.path.exists(img_path):
                continue
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            _, img = cv2.threshold(img, seuil, 255, cv2.THRESH_BINARY)
            img = img.astype(np.float32)
            _, img_rot = rotate(img, xy_anche)
            img_rogn =  rogner(img_rot, xy_anche)
            
            Xv , Yv = deplacement_profil(img_rogn, seuil, coord_rogn)
            
            plot_fig = False
            if plot_fig :
                plt.figure()
                plt.imshow(img_rogn, cmap='gray')
                plt.plot(Xv, Yv, 'm.-', linewidth=0.1)

            # file.write(str(dim_canal) + '\n')
    

C:\Users\Admin\AppData\Local\Temp\ipykernel_14824\255901586.py:30: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
